In [ ]:
import json

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

from sentinelhub import SHConfig

from source.utils import CLIENT_ID, CLIENT_SECRET

# Fetch token and authenticate

In [173]:
# set up credentials
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)

# get an authentication token
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
                          client_secret=CLIENT_SECRET, include_client_id=True)

# All requests using this session will have an access token automatically added
# resp = oauth.get("https://services.sentinel-hub.com/configuration/v1/wms/instances")


# Another way to authenticate

In [28]:
config = SHConfig()

config.sh_client_id = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET

if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Statistical API, please provide the credentials (OAuth client ID and client secret).")

# Set geometry

In [ ]:
geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -83.60084,
        8.753013
      ],
      [
        -83.599982,
        8.748644
      ],
      [
        -83.596335,
        8.749641
      ],
      [
        -83.595866,
        8.752738
      ],
      [
        -83.60084,
        8.753013
      ]
    ]
  ]
}

# Set evalscript 

In [ ]:
evalscript = """
//VERSION=3
// To set custom max and min values, set
// defaultVis to false and choose your max and
// min values. The color map will then be scaled
// to those max and min values

// LST has two observations per days: 1h30 and 13h30 solar local time

const defaultVis = true; // true or false
const color_min = 263; // default min: 263
const color_max = 340; // default max: 340
const sensing_time = "13"; // "0130" or "1330" or ""

//set the data for map and timeserie
function setup() {
    return {
        input: ["LST", "dataMask"],
        output: [
            //{ id: "default", bands: 4 },
            //{ id: "index", bands: 1, sampleType: "FLOAT32" },
            { id: "eobrowserStats", bands: 1, sampleType: "FLOAT32" },
            { id: "dataMask", bands: 1 }
        ],
        mosaicking: "TILE"
    };
}

//Select files based on sensing time (0130 or 1330)
function preProcessScenes(collections) {
    collections.scenes.tiles = collections.scenes.tiles.filter(function (tile) {
        return tile.dataPath.includes("T" + sensing_time);
    });
    return collections;
}

function evaluatePixel(samples) {
    // LST scale factor
    const scaleFactor = 100;

    // use the first sample with a datamask of 1
    datamask = 0;
    val = 0/0;
    for (var i = 0; i < samples.length; i++) {
        datamask = samples[i].dataMask;
        if (datamask == 1) {
            val = samples[i].LST / scaleFactor;
            break;
        }
    }    
    
    return {
        //default: [...visualizer.process(val), datamask],
        //index: [val],
        eobrowserStats: [val, datamask],
        dataMask: [datamask],
    };
}
"""

# Bild JSON request

In [ ]:
json_request = {
  "input": {
    "bounds": {
      "geometry": geometry
    },
    "data": [
      {
        "dataFilter": {},
        "type": "byoc-723331c0-74d4-464a-8e0f-2433cb07c1aa" # custom data set
      }
    ]
  },
  "aggregation": {
    "timeRange": { # the time interval for the data
      "from": "2023-01-01T00:00:00Z",
      "to": "2023-02-01T23:59:59Z"
    },
    "aggregationInterval": {
      "of": "P10D" # day interval 
    },
    "width": None,
    "height": None,
    "evalscript": evalscript
  },
  "calculations": {
    "default": {}
  }
}

# Request data 

In [124]:
# Set the request url and headers
url = "https://services.sentinel-hub.com/api/v1/statistics"
headers = {
  # "Authorization": "Bearer <YOUR_TOKEN_HERE>",
  "Accept": "application/json",
  "Content-Type": "application/json"
}

#Send the request
response = oauth.request(
    "POST", url, headers=headers, json = json_request
)

In [ ]:
result = response.json()
print(json.dumps(result, indent=2))